Este archivo descarga el dataset COCO 2017 (validación) y genera un subconjunto con las primeras 100 imágenes y sus anotaciones, tal como se especifica en el enunciado.

In [1]:
import requests
import zipfile
import json
import shutil
import os
from pathlib import Path

# Directorio actual donde está el notebook
root = Path.cwd()

# Rutas temporales para descarga
temp_dir = root / "temp_coco"
temp_dir.mkdir(exist_ok=True)

# Rutas finales del dataset
dataset_dir = root / "dataset"
img_dir = dataset_dir / "val2017"
ann_dir = dataset_dir / "annotations"

# URLs oficiales de COCO 2017
URL_IMAGES = "http://images.cocodataset.org/zips/val2017.zip"
URL_ANNOTATIONS = "http://images.cocodataset.org/annotations/annotations_trainval2017.zip"

In [3]:
def download_and_extract(url, destination):
    filename = url.split('/')[-1]
    zip_path = temp_dir / filename
    
    if not zip_path.exists():
        print(f"Descargando {filename}...")
        with requests.get(url, stream=True) as r:
            r.raise_for_status()
            with open(zip_path, 'wb') as f:
                for chunk in r.iter_content(chunk_size=8192):
                    f.write(chunk)
    
    print(f"Extrayendo {filename}...")
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(temp_dir)

# Ejecutar descargas
download_and_extract(URL_ANNOTATIONS, temp_dir)
download_and_extract(URL_IMAGES, temp_dir)

Extrayendo annotations_trainval2017.zip...
Extrayendo val2017.zip...


In [4]:
# Rutas de los archivos extraídos en temp
raw_ann_file = temp_dir / "annotations" / "instances_val2017.json"
raw_img_dir = temp_dir / "val2017"

# Crear estructura final
img_dir.mkdir(parents=True, exist_ok=True)
ann_dir.mkdir(parents=True, exist_ok=True)

limit = 100

print(f"Generando subconjunto de las primeras {limit} imagenes...")

if raw_img_dir.exists():
    # Listar y ordenar imágenes descargadas
    images = sorted([f for f in os.listdir(raw_img_dir) if f.endswith(('.jpg', '.jpeg', '.png'))])
    
    # Seleccionar las primeras 100
    selected_images = images[:limit]
    print(f"Se han encontrado {len(images)} imágenes. Seleccionando {len(selected_images)}...")

    # Copiar imágenes seleccionadas
    for img_name in selected_images:
        src = raw_img_dir / img_name
        dst = img_dir / img_name
        if not dst.exists():
            shutil.copy2(src, dst)
    print(f"Imágenes copiadas a {img_dir}")

    # Copiar archivo de anotaciones
    if raw_ann_file.exists():
        dst_ann = ann_dir / raw_ann_file.name
        if not dst_ann.exists():
            shutil.copy2(raw_ann_file, dst_ann)
        print(f"Anotaciones copiadas a {ann_dir}")
        
    print("¡Proceso completado!")
else:
    print(f"No se encontró el directorio {raw_img_dir}. Asegúrate de que la descarga finalizó correctamente.")

Generando subconjunto de las primeras 100 imagenes...
Se han encontrado 5000 imágenes. Seleccionando 100...
Imágenes copiadas a c:\Users\usuario\OneDrive\Desktop\AprendizajeAutomatico\practicas-aprendizaje\profundo\dataset\val2017
Anotaciones copiadas a c:\Users\usuario\OneDrive\Desktop\AprendizajeAutomatico\practicas-aprendizaje\profundo\dataset\annotations
¡Proceso completado!


In [5]:
# Elimina el directorio creado temporalmente

if os.path.exists(temp_dir):
    try:
        shutil.rmtree(temp_dir)
        print(f"Directorio {temp_dir} eliminado exitosamente.")
    except Exception as e:
        print(f"Error al eliminar el directorio {temp_dir}: {str(e)}")
else:
    print(f"El directorio {temp_dir} no existe.")


Directorio c:\Users\usuario\OneDrive\Desktop\AprendizajeAutomatico\practicas-aprendizaje\profundo\temp_coco eliminado exitosamente.
